In [43]:
import torch
import pandas as pd
import re
import pickle
import emoji
import preprocessor
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

In [44]:
data = pd.read_csv('raw_data/tweets.csv')

In [45]:
df = data.drop(columns=['media', 'inReplyToUser', 'mentionedUsers', 'lang', 'source', 'location'])
df.rename(columns={'rawContent': 'tweet'}, inplace=True)

In [46]:
def convert_to_datetime(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    return df

In [47]:
def convert_emojis(string):
    text = emoji.demojize(string)
    text = text.replace(":", "").replace("_", " ")
    return text

In [64]:
def clean_tweet(tweet):
    
    # Remove url
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)
    tweet = re.sub(r'http?:\/\/\S+', '', tweet)
    
    # Remove \n
    tweet = re.sub(r'\n', '', tweet)
    
    # Remove @
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)
    
    # Remove #
    tweet = re.sub(r'#', '', tweet)
    
    # Remove RT
    tweet = re.sub(r'RT[\s]+', '', tweet)
    
    # Convert to lower case
    tweet = tweet.lower()
    
    # Remove "#&'()*/:;@[\]^`{|}~"
    tweet = re.sub(r'[#&\'\(\)\*\+\/:;@\[\]\^`{|}~]', '', tweet)
    
    # Remove double space
    tweet = re.sub(r'\s\s+', ' ', tweet)
    
    return tweet

In [49]:
def min_max_normalization(df):
    df = df.copy()
    
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    scaler = MinMaxScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])
    
    return df

In [50]:
df = convert_to_datetime(df)
df['tweet'] = df['tweet'].apply(clean_tweet)
df['tweet'] = df['tweet'].apply(convert_emojis)
df = min_max_normalization(df)

In [52]:
list1 = ['AAPL', 'Apple Inc.', 'Appl', 'AMC', 'AMC Entertainment Holdings Inc.', 'AAMC', 'AC', 'AMZN', 'Amazon.com Inc.', 'Amazn', 'AMD', 'Advanced Micro Devices Inc.', 'AMED', 'BB', 'BlackBerry Ltd.', 'Blacberry', 'BBBY', 'Bed Bath & Beyond Inc.', 'Bedbathbeyond', 'BTC', 'Bitcoin', 'bit coin', 'BYND', 'Beyond Meat Inc.', 'Bynd', 'CLNE', 'Clean Energy Fuels Corp.', 'Clnenergy', 'CMG', 'Chipotle Mexican Grill Inc.', 'Chipotle', 'COST', 'Costco Wholesale Corp.', 'Coscto', 'CRSR', 'Corsair Gaming Inc.', 'Corsair', 'DIS', 'Walt Disney Co.', 'Disney', 'DOGE', 'Dogecoin', 'Doge coin', 'ET', 'Energy Transfer LP', 'Energy Transfer', 'F', 'Ford Motor Co.', 'Ford', 'FB', 'Facebook Inc.', 'Fb', 'GME', 'GameStop Corp.', 'Gamestop', 'GOOG', 'Alphabet Inc.', 'Google', 'HD', 'Home Depot Inc.', 'Home Depo', 'INTC', 'Intel Corp.', 'Intell', 'JNJ', 'Johnson & Johnson', 'Johnsohn', 'KO', 'Coca-Cola Co.', 'Coca Cola', 'LULU', 'Lululemon Athletica Inc.', 'Lululemn', 'MCD', "McDonald's Corp.", "McDonalds", 'MGM', 'MGM Resorts International', 'Mgmresorts', 'MSFT', 'Microsoft Corp.', 'Micrsoft', 'MU', 'Micron Technology Inc.', 'Micron', 'NIO', 'NIO Inc.', 'Nio', 'NVDA', 'NVIDIA Corp.', 'Nivida', 'PFE', 'Pfizer Inc.', 'Pifzer', 'PINS', 'Pinterest Inc.', 'Pintrst', 'PLTR', 'Palantir Technologies Inc.', 'Palintir', 'QQQ', 'Invesco QQQ Trust', 'InvescoQQQ', 'RBLX', 'Roblox Corp.', 'Robloks', 'RIOT', 'Riot Blockchain Inc.', 'Riott', 'ROKU', 'Roku Inc.', 'Rokue', 'SNDL', 'Sundial Growers Inc.', 'Sundail', 'SPCE', 'Virgin Galactic Holdings Inc.', 'Spacex', 'SQ', 'Square Inc.', 'Squar', 'T', 'AT&T Inc.', 'Att', 'TSLA', 'Tesla Inc.', 'Teslla', 'TWTR', 'Twitter Inc.', 'Twiter', 'UBER', 'Uber Technologies Inc.', 'Ube', 'UPST', 'Upstart Holdings Inc.', 'Upstartholdings', 'V', 'Visa Inc.', 'Visa', 'WMT', 'Walmart Inc.', 'Walmrt', 'XOM', 'Exxon Mobil Corp.', 'Exxonmobil']
list2 = ['AAL', 'American Airlines Group Inc.', 'Americanairlines',
        'ABNB', 'Airbnb Inc.', 'AirBnB',
        'ACB', 'Aurora Cannabis Inc.', 'Auroracannabis',
        'AMRN', 'Amarin Corp. plc', 'Amerin',
        'ARKK', 'ARK Innovation ETF', 'ARKinovation',
        'BABA', 'Alibaba Group Holding Ltd.', 'Alibaba',
        'BA', 'Boeing Co.', 'Boing',
        'BAC', 'Bank of America Corp.', 'Bankofamerica',
        'BIDU', 'Baidu Inc.', 'Bido',
        'BILI', 'Bilibili Inc.', 'Billibili',
        'BLNK', 'Blink Charging Co.', 'Blinkcharg',
        'BMY', 'Bristol Myers Squibb Co.', 'Bristolmyers',
        'BRK.A', 'Berkshire Hathaway Inc.', 'Berkshira',
        'CCL', 'Carnival Corp.', 'Carnival',
        'CGC', 'Canopy Growth Corp.', 'Canopygrowth',
        'CHWY', 'Chewy Inc.', 'Chewycom',
        'CSCO', 'Cisco Systems Inc.', 'Cicsco',
        'CVS', 'CVS Health Corp.', 'Cvshealth',
        'DAL', 'Delta Air Lines Inc.', 'Deltaairlines',
        'DDOG', 'Datadog Inc.', 'DataDog',
        'DISCA', 'Discovery Inc. - Class A', 'Discovera',
        'DKNG', 'DraftKings Inc.', 'DraftKings',
        'ENPH', 'Enphase Energy Inc.', 'Enphase',
        'EQT', 'EQT Corp.', 'Eqtcorporation',
        'FCEL', 'FuelCell Energy Inc.', 'Fuelcell',
        'FSLY', 'Fastly Inc.', 'Fastly',
        'GE', 'General Electric Co.', 'GeneralElectric',
        'GM', 'General Motors Co.', 'GeneralMotors',
        'GOLD', 'Barrick Gold Corp.', 'Goldmining',
        'GPRO', 'GoPro Inc.', 'Gopro',
        'GRWG', 'GrowGeneration Corp.', 'Growgen',
        'HPE', 'Hewlett Packard Enterprise Co.', 'Hewlettpackard',
        'IBB', 'iShares NASDAQ Biotechnology ETF', 'iSharesNasdaqBiotechnology',
        'IBKR', 'Interactive Brokers Group Inc.', 'InterectiveBrokers',
        'INO', 'Inovio Pharmaceuticals Inc.', 'Inovio',
        'JD', 'JD.com Inc.', 'JD',
        'JMIA', 'Jumia Technologies AG', 'Jumia',
        'JPM', 'JPMorgan Chase & Co.', 'JPmorgan',
        'KHC', 'Kraft Heinz Co.', 'KraftHeinz',
        'LIT', 'Global X Lithium & Battery Tech ETF', 'GlobalXlithium',
        'MARA', 'Marathon Digital Holdings Inc.', 'MarathonDigital',
        'MRNA', 'Moderna Inc.', 'Modernna',
        'NCLH', 'Norwegian Cruise Line Holdings Ltd.', 'Norwegiancruiseline',
        'NET', 'Cloudflare Inc.', 'CloudFlare',
        'NFLX', 'Netflix Inc.', 'Netlfix',
        'NKE', 'Nike Inc.', 'Nike', 'NKLA', 'Nikola Corp.', 'Nicola', 'NOK', 'Nokia Corp.', 'Noka', 'O', 'Realty Income Corp.', 'Realtyincome', 'OGI', 'OrganiGram Holdings Inc.', 'Organigram', 'OTRK', 'Ontrak Inc.', 'OnTrack', 'PDD', 'Pinduoduo Inc.', 'Pinduoduo', 'PENN', 'Penn National Gaming Inc.', 'PennNational']


In [53]:
keywords = []

for word in list1:
    word = clean_tweet(word)
    word = word.replace('inc.', '').replace('ltd.', '').replace('co.', '').replace('corp.', '').replace('.com', '')
    keywords.append(word)

for word in list2:
    word = clean_tweet(word)
    word = word.replace('inc.', '').replace('ltd.', '').replace('co.', '').replace('corp.', '').replace('.com', '')
    keywords.append(word)

In [54]:
top50 = pd.read_csv('raw_data/top50.csv')

In [55]:
cashtags = top50['cashtags'].tolist()
hashtags = top50['hashtags'].tolist()

In [56]:
for cashtag in cashtags:
    cashtag = clean_tweet(cashtag)
    cashtag = re.sub(r'[^\w\s]', '', cashtag)
    cashtag = cashtag.replace(' ', '')
    
    if cashtag not in keywords:
        keywords.append(cashtag)

In [57]:
for hashtag in hashtags:
    hashtag = clean_tweet(hashtag)
    hashtag = re.sub(r'[^\w\s]', '', hashtag)
    hashtag = hashtag.replace(' ', '')
    
    if hashtag not in keywords:
        keywords.append(hashtag)

In [58]:
for keyword in keywords:
    if len(keyword) < 2:
        keywords.remove(keyword)

In [59]:
df_filtered = df[df['tweet'].str.contains(keyword).any() | df['cashtags'].notnull() | df['hashtags'].notnull()]

In [63]:
df_filtered

,url,date,tweet,username,likeCount,replyCount,retweetCount,quoteCount,cashtags,hashtags,viewCount,sentiment
11,https://twitter.com/unusual_whales/status/1608...,2022-12-30 20:25:41+00:00,$spys intraday flow overview right now link li...,unusual_whales,0.000386,0.000479,0.000391,0.000000,['SPY'],NaN,0.006192,Neutral
13,https://twitter.com/unusual_whales/status/1608...,2022-12-30 20:00:21+00:00,covid-19 issues in china are still hampering m...,unusual_whales,0.000874,0.001436,0.000313,0.000246,['AAPL'],NaN,0.005284,Neutral
15,https://twitter.com/unusual_whales/status/1608...,2022-12-30 19:53:56+00:00,warren buffett of $brk.b has surpassed jeff be...,unusual_whales,0.002999,0.003230,0.001055,0.000185,"['BRK.B', 'AMZN']",NaN,0.009775,Neutral
16,https://twitter.com/unusual_whales/status/1608...,2022-12-30 19:51:16+00:00,here are all the current opening flow alerts 1...,unusual_whales,0.000185,0.000479,0.000000,0.000000,"['SNOW', 'PANW', 'SI', 'JD']",NaN,0.005625,Neutral
18,https://twitter.com/unusual_whales/status/1608...,2022-12-30 19:30:59+00:00,price action for $tsla today for full charting...,unusual_whales,0.000277,0.001316,0.000352,0.000000,['TSLA'],NaN,0.006668,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
20394,https://twitter.com/unusual_whales/status/1511...,2022-04-06 11:15:00+00:00,"$gme flow- putcall ratio 0.577, call premium 3...",unusual_whales,0.000336,0.000598,0.000195,0.000062,['GME'],NaN,NaN,Neutral
20395,https://twitter.com/unusual_whales/status/1510...,2022-04-04 14:58:06+00:00,let’s look at politicians and $twtr.no one bou...,unusual_whales,0.001008,0.000957,0.000195,0.000123,['TWTR'],NaN,NaN,Neutral
20396,https://twitter.com/unusual_whales/status/1509...,2022-04-01 15:05:46+00:00,"$qqq 2022-04-22 c $396underlying $361.51, % di...",unusual_whales,0.000193,0.000120,0.000078,0.000000,['QQQ'],NaN,NaN,Neutral
20398,https://twitter.com/unusual_whales/status/1509...,2022-03-31 14:30:36+00:00,"$spy 2022-04-29 p $395underlying $457.12, % di...",unusual_whales,0.000160,0.000479,0.000039,0.000000,['SPY'],NaN,NaN,Neutral


In [61]:
def sentiment_analysis(df):
    
    from transformers import BertTokenizer, BertForSequenceClassification, pipeline

    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
    nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
    
    for i in range(len(df)):
        results = nlp(df.iloc[i]['tweet'])
        df['sentiment'] = results[0]['label']
    
    return df
  

In [62]:
sentiment_analysis(df_filtered)

/tmp/ipykernel_5166/3005844282.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = results[0]['label']


KeyboardInterrupt: 